In [2]:
#%pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu124 --force-reinstall

In [3]:
"""import torch

def format_pytorch_version(version_str):
    # Example input: "2.0.1+cu118" -> returns "2.0.1"
    return version_str.split('+')[0]

def format_cuda_version(cuda_str):
    # If CUDA is None (CPU-only PyTorch), return "cpu"
    if cuda_str is None:
        return "cpu"
    # Example: "11.8" -> "cu118"
    return "cu" + cuda_str.replace('.', '')

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)
CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

print(f"Installing packages for torch-{TORCH}+{CUDA}...")

%pip install --upgrade --no-cache-dir pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html

%pip install torch_geometric"""

'import torch\n\ndef format_pytorch_version(version_str):\n    # Example input: "2.0.1+cu118" -> returns "2.0.1"\n    return version_str.split(\'+\')[0]\n\ndef format_cuda_version(cuda_str):\n    # If CUDA is None (CPU-only PyTorch), return "cpu"\n    if cuda_str is None:\n        return "cpu"\n    # Example: "11.8" -> "cu118"\n    return "cu" + cuda_str.replace(\'.\', \'\')\n\nTORCH_version = torch.__version__\nTORCH = format_pytorch_version(TORCH_version)\nCUDA_version = torch.version.cuda\nCUDA = format_cuda_version(CUDA_version)\n\nprint(f"Installing packages for torch-{TORCH}+{CUDA}...")\n\n%pip install --upgrade --no-cache-dir pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html\n\n%pip install torch_geometric'

In [4]:
#%pip install scikit-learn imbalanced-learn xgboost

In [5]:
#%pip install optuna

### Hyperparameter Optimization Citeseer
#### SCAR 

In [3]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "citeseer",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 2, 9),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.09, 0.18),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs": trial.suggest_int("rate_pairs", 1, 15),
        "batch_size": trial.suggest_categorical("batch_size", [5,10,15,20]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":"nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "citeseer_scar_sampling_nnif_batch_cluster.csv",
        "min":0.845,
        "clusters":trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":100,         
        "sampling":"cluster",#trial.suggest_categorical("sampling",["sage","cluster","shine"]),         
        
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-04-02 21:48:33,519] A new study created in memory with name: no-name-aff14e9e-fd76-413d-93dc-e4c3d5b6fb50
Computing METIS partitioning...
Done!


Running experiment with seed=654:
 - K=7, layers=1, hidden=256, out=256
 - norm=None, dropout=0, batch_size=20, methodology=ours
 - ratio=0.09053295232680494, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=cluster
 - model_type=GCNConv, rate_pairs=7, clusters=200, lr=0.003274224268094839
Epoch 0 / 100, Loss: 10.0654
Epoch 10 / 100, Loss: 8.5717
Epoch 20 / 100, Loss: 7.6884
Epoch 30 / 100, Loss: 6.6335
Epoch 40 / 100, Loss: 5.4427
Epoch 50 / 100, Loss: 4.5829
Epoch 60 / 100, Loss: 3.6111
Epoch 70 / 100, Loss: 2.8355
Epoch 80 / 100, Loss: 2.4004
Epoch 90 / 100, Loss: 1.9723
 - Metrics: Accuracy=0.9411, F1=0.8497, Recall=0.7903, Precision=0.9187
Running experiment with seed=114:
 - K=7, layers=1, hidden=256, out=256
 - norm=None, dropout=0, batch_size=20, methodology=ours
 - ratio=0.09053295232680494, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=cluster
 - model_type=GCNConv, rate_pairs=7, clusters=200, lr=0.003274224

Computing METIS partitioning...
Done!


Epoch 0 / 100, Loss: 10.1618
Epoch 10 / 100, Loss: 8.9259
Epoch 20 / 100, Loss: 7.9734
Epoch 30 / 100, Loss: 6.7274
Epoch 40 / 100, Loss: 5.4196
Epoch 50 / 100, Loss: 4.5117
Epoch 60 / 100, Loss: 3.4844
Epoch 70 / 100, Loss: 2.8126
Epoch 80 / 100, Loss: 2.2958
Epoch 90 / 100, Loss: 1.9424


[I 2025-04-02 21:50:51,703] Trial 0 finished with value: 0.8448925324233364 and parameters: {'K': 7, 'layers': 1, 'hidden_channels': 256, 'out_channels': 256, 'ratio': 0.09053295232680494, 'model_type': 'GCNConv', 'rate_pairs': 7, 'batch_size': 20, 'lr': 0.003274224268094839, 'clusters': 200}. Best is trial 0 with value: 0.8448925324233364.


...

Epoch 0 / 100, Loss: 2.1124
Epoch 10 / 100, Loss: 1.7860
Epoch 20 / 100, Loss: 1.6003
Epoch 30 / 100, Loss: 1.4671
Epoch 40 / 100, Loss: 1.3257
Epoch 50 / 100, Loss: 1.1682
Epoch 60 / 100, Loss: 1.0581
Epoch 70 / 100, Loss: 0.9314
Epoch 80 / 100, Loss: 0.8580
Epoch 90 / 100, Loss: 0.7694
 - Metrics: Accuracy=0.9402, F1=0.8584, Recall=0.8602, Precision=0.8565
Running experiment with seed=281:
 - K=3, layers=1, hidden=256, out=256
 - norm=None, dropout=0, batch_size=20, methodology=ours
 - ratio=0.164569840201006, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=cluster
 - model_type=GCNConv, rate_pairs=4, clusters=50, lr=0.0055187924923012525


Computing METIS partitioning...
Done!


Epoch 0 / 100, Loss: 1.9995
Epoch 10 / 100, Loss: 1.6955
Epoch 20 / 100, Loss: 1.5959
Epoch 30 / 100, Loss: 1.4673
Epoch 40 / 100, Loss: 1.2776
Epoch 50 / 100, Loss: 1.1728
Epoch 60 / 100, Loss: 1.0089
Epoch 70 / 100, Loss: 0.9582
Epoch 80 / 100, Loss: 0.8187
Epoch 90 / 100, Loss: 0.7359


[I 2025-04-03 03:58:06,965] Trial 199 finished with value: 0.8624308852561604 and parameters: {'K': 3, 'layers': 1, 'hidden_channels': 256, 'out_channels': 256, 'ratio': 0.164569840201006, 'model_type': 'GCNConv', 'rate_pairs': 4, 'batch_size': 20, 'lr': 0.0055187924923012525, 'clusters': 50}. Best is trial 183 with value: 0.8654470713144544.


 - Metrics: Accuracy=0.9462, F1=0.8719, Recall=0.8688, Precision=0.8750
Done. Results written to citeseer_experimentations\citeseer_scar_sampling_nnif_batch_cluster_0304035602.csv.
Average F1 over valid seeds: 0.8624 ± 0.0050
Best trial:
  Average F1: 0.8654470713144544
  Best parameters:
    K: 3
    layers: 1
    hidden_channels: 256
    out_channels: 256
    ratio: 0.16853471256009084
    model_type: GCNConv
    rate_pairs: 4
    batch_size: 20
    lr: 0.0052668287985801095
    clusters: 50


### Hyperparameter Optimization Citeseer
#### SAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "citeseer",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 2, 7),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.09, 0.18),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":trial.suggest_int("layers", 1, 15),
        "batch_size": trial.suggest_categorical("batch_size", [256,512,1024]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "citeseer_sar_sampling_nnif_batch_cluster.csv",
        "min":0.845,
        "clusters":trial.suggest_categorical("clusters", [50,100,200,300,400]),
        "num_epochs":100,         
        "sampling":trial.suggest_categorical("sampling",["sage","cluster","shine"]),         
        
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-03-25 10:10:43,640] A new study created in memory with name: no-name-f25fde7d-ee4a-4fa2-be4f-f9e486b8970e


Running experiment with seed=654:
 - K=3, layers=1, hidden=256, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.11307195638376069, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=75
 - model_type=GATConv, rate_pairs=1, clusters=50, lr=0.0023234250431828636
Epoch 0 / 100, Loss: 2.8385
Epoch 10 / 100, Loss: 2.4108
Epoch 20 / 100, Loss: 2.2628
Epoch 30 / 100, Loss: 2.1825
Epoch 40 / 100, Loss: 2.0409
Epoch 50 / 100, Loss: 1.9945
Epoch 60 / 100, Loss: 1.8232
Epoch 70 / 100, Loss: 1.6971
Epoch 80 / 100, Loss: 1.6468
Epoch 90 / 100, Loss: 1.4848
 - Metrics: Accuracy=0.9405, F1=0.8507, Recall=0.8046, Precision=0.9024
Running experiment with seed=114:
 - K=3, layers=1, hidden=256, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.11307195638376069, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=75
 - model_type=GATConv, rate_pa

[I 2025-03-25 10:13:49,885] Trial 0 finished with value: 0.857595508989174 and parameters: {'K': 3, 'layers': 1, 'hidden_channels': 256, 'out_channels': 256, 'ratio': 0.11307195638376069, 'model_type': 'GATConv', 'batch_size': 1024, 'lr': 0.0023234250431828636, 'anomaly_detector': 'nearest_neighbors', 'clusters': 50, 'num_epochs': 100, 'sampling': 'neighbor', 'sampling_k': 75}. Best is trial 0 with value: 0.857595508989174.


 - Metrics: Accuracy=0.9390, F1=0.8493, Recall=0.8160, Precision=0.8854
Done. Results written to citeseer_experimentations\citeseer_sar_sampling_nnif_batch_cluster_2503101043.csv.
Average F1 over valid seeds: 0.8576 ± 0.0077
Running experiment with seed=654:
 - K=7, layers=2, hidden=256, out=64
 - norm=None, dropout=0, batch_size=256, methodology=ours
 - ratio=0.17225386531043968, aggregation=mean, treatment=removal, anomaly_detector=unweighted, sampling=nearest_neighbors, sampling_k=25
 - model_type=GATConv, rate_pairs=2, clusters=400, lr=0.008752505664905935
Epoch 0 / 150, Loss: 42.4167
Epoch 10 / 150, Loss: 18.6557
Epoch 20 / 150, Loss: 10.0401
Epoch 30 / 150, Loss: 6.0609
Epoch 40 / 150, Loss: 4.2251
Epoch 50 / 150, Loss: 3.0423
Epoch 60 / 150, Loss: 2.2833
Epoch 70 / 150, Loss: 1.8259
Epoch 80 / 150, Loss: 1.4801
Epoch 90 / 150, Loss: 1.2157
Epoch 100 / 150, Loss: 1.0309
Epoch 110 / 150, Loss: 0.8732
Epoch 120 / 150, Loss: 0.7590
Epoch 130 / 150, Loss: 0.6525
Epoch 140 / 150, Loss

[I 2025-03-25 10:15:40,677] Trial 1 finished with value: 0.6813996316758748 and parameters: {'K': 7, 'layers': 2, 'hidden_channels': 256, 'out_channels': 64, 'ratio': 0.17225386531043968, 'model_type': 'GATConv', 'batch_size': 256, 'lr': 0.008752505664905935, 'anomaly_detector': 'unweighted', 'clusters': 400, 'num_epochs': 150, 'sampling': 'nearest_neighbors', 'sampling_k': 25}. Best is trial 0 with value: 0.857595508989174.


...

 - Metrics: Accuracy=0.9059, F1=0.7183, Recall=0.5692, Precision=0.9732
F1 = 0.72 < 0.845, skipping ...
Done. Results written to citeseer_experimentations\citeseer_sar_sampling_nnif_batch_cluster_2503140907.csv.
Average F1 over valid seeds: 0.7183 ± 0.0000
Running experiment with seed=654:
 - K=2, layers=1, hidden=256, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.15841141409782936, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=25
 - model_type=GCNConv, rate_pairs=1, clusters=400, lr=0.004733199465670689
Epoch 0 / 100, Loss: 2.1699
Epoch 10 / 100, Loss: 1.6532
Epoch 20 / 100, Loss: 1.4729
Epoch 30 / 100, Loss: 1.3237
Epoch 40 / 100, Loss: 1.1338
Epoch 50 / 100, Loss: 1.0336
Epoch 60 / 100, Loss: 0.8867
Epoch 70 / 100, Loss: 0.7703
Epoch 80 / 100, Loss: 0.7087
Epoch 90 / 100, Loss: 0.6093
 - Metrics: Accuracy=0.9408, F1=0.8600, Recall=0.8631, Precision=0.8569
Running experiment with seed=114:
 - K=2

[I 2025-03-25 14:12:40,226] Trial 98 finished with value: 0.8617602498928539 and parameters: {'K': 2, 'layers': 1, 'hidden_channels': 256, 'out_channels': 256, 'ratio': 0.15841141409782936, 'model_type': 'GCNConv', 'batch_size': 1024, 'lr': 0.004733199465670689, 'anomaly_detector': 'nearest_neighbors', 'clusters': 400, 'num_epochs': 100, 'sampling': 'neighbor', 'sampling_k': 25}. Best is trial 87 with value: 0.8650138572197479.


 - Metrics: Accuracy=0.9387, F1=0.8539, Recall=0.8502, Precision=0.8576
Done. Results written to citeseer_experimentations\citeseer_sar_sampling_nnif_batch_cluster_2503140941.csv.
Average F1 over valid seeds: 0.8618 ± 0.0066
Running experiment with seed=654:
 - K=2, layers=1, hidden=256, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.16102039029803536, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=nearest_neighbors, sampling_k=25
 - model_type=GCNConv, rate_pairs=1, clusters=400, lr=0.004605584968207975
Epoch 0 / 100, Loss: 4.1609
Epoch 10 / 100, Loss: 3.4473
Epoch 20 / 100, Loss: 3.0725
Epoch 30 / 100, Loss: 2.6641
Epoch 40 / 100, Loss: 2.3129
Epoch 50 / 100, Loss: 2.1049
Epoch 60 / 100, Loss: 1.7786
Epoch 70 / 100, Loss: 1.6623
Epoch 80 / 100, Loss: 1.4632
Epoch 90 / 100, Loss: 1.3323


[I 2025-03-25 14:13:15,258] Trial 99 finished with value: 0.8429629629629629 and parameters: {'K': 2, 'layers': 1, 'hidden_channels': 256, 'out_channels': 256, 'ratio': 0.16102039029803536, 'model_type': 'GCNConv', 'batch_size': 1024, 'lr': 0.004605584968207975, 'anomaly_detector': 'nearest_neighbors', 'clusters': 400, 'num_epochs': 100, 'sampling': 'nearest_neighbors', 'sampling_k': 25}. Best is trial 87 with value: 0.8650138572197479.


 - Metrics: Accuracy=0.9363, F1=0.8430, Recall=0.8117, Precision=0.8767
F1 = 0.84 < 0.845, skipping ...
Done. Results written to citeseer_experimentations\citeseer_sar_sampling_nnif_batch_cluster_2503141240.csv.
Average F1 over valid seeds: 0.8430 ± 0.0000
Best trial:
  Average F1: 0.8650138572197479
  Best parameters:
    K: 2
    layers: 1
    hidden_channels: 128
    out_channels: 256
    ratio: 0.1611386183555754
    model_type: GCNConv
    batch_size: 1024
    lr: 0.0063212510924867535
    anomaly_detector: nearest_neighbors
    clusters: 200
    num_epochs: 100
    sampling: neighbor
    sampling_k: 25


### Hyperparameter Optimization Cora
#### SCAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "cora",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 1, 12),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.16, 0.34),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":trial.suggest_int("rate_pairs", 1, 15),
        "batch_size": trial.suggest_categorical("batch_size", [5,10,15,20]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":"nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "cora_scar_sampling_nnif_batch_cluster.csv",
        "min":0.85,
        "clusters":trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":100,         
        "sampling":"cluster",#trial.suggest_categorical("sampling",["sage","cluster","shine"]),         
        
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-03-27 10:52:51,733] A new study created in memory with name: no-name-0e26df09-7fa5-46d1-be19-f32a53fd7c0d


Running experiment with seed=654:
 - K=10, layers=1, hidden=64, out=64
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.19622450292840055, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=50
 - model_type=GCNConv, rate_pairs=1, clusters=400, lr=0.003767773886669508
Epoch 0 / 100, Loss: 8.2044
Epoch 10 / 100, Loss: 7.3760
Epoch 20 / 100, Loss: 6.8701
Epoch 30 / 100, Loss: 6.4718
Epoch 40 / 100, Loss: 5.6112
Epoch 50 / 100, Loss: 4.9094
Epoch 60 / 100, Loss: 4.1198
Epoch 70 / 100, Loss: 3.4837
Epoch 80 / 100, Loss: 2.9254
Epoch 90 / 100, Loss: 2.5098
 - Metrics: Accuracy=0.9232, F1=0.8706, Recall=0.8557, Precision=0.8861
Running experiment with seed=114:
 - K=10, layers=1, hidden=64, out=64
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.19622450292840055, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=50
 - model_type=GCNConv, rate_pairs=

[I 2025-03-27 10:55:58,397] Trial 0 finished with value: 0.8818876658327218 and parameters: {'K': 10, 'layers': 1, 'hidden_channels': 64, 'out_channels': 64, 'ratio': 0.19622450292840055, 'model_type': 'GCNConv', 'batch_size': 512, 'lr': 0.003767773886669508, 'clusters': 400, 'sampling': 'neighbor', 'sampling_k': 50}. Best is trial 0 with value: 0.8818876658327218.


 - Metrics: Accuracy=0.9343, F1=0.8882, Recall=0.8643, Precision=0.9134
Done. Results written to cora_experimentations\cora_scar_sampling_nnif_batch_cluster_2703105251.csv.
Average F1 over valid seeds: 0.8819 ± 0.0069
Running experiment with seed=654:
 - K=8, layers=1, hidden=128, out=256
 - norm=None, dropout=0, batch_size=15, methodology=ours
 - ratio=0.22715757417672017, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=nearest_neighbors, sampling_k=25
 - model_type=GCNConv, rate_pairs=1, clusters=200, lr=0.008027462923101808
Epoch 0 / 100, Loss: 443.7636
Epoch 10 / 100, Loss: 9.9173
Epoch 20 / 100, Loss: 2.5276
Epoch 30 / 100, Loss: 0.8766
Epoch 40 / 100, Loss: 0.3332
Epoch 50 / 100, Loss: 0.1330
Epoch 60 / 100, Loss: 0.0536
Epoch 70 / 100, Loss: 0.0216
Epoch 80 / 100, Loss: 0.0088
Epoch 90 / 100, Loss: 0.0035


[I 2025-03-27 11:05:52,536] Trial 1 finished with value: 0.7062359128474831 and parameters: {'K': 8, 'layers': 1, 'hidden_channels': 128, 'out_channels': 256, 'ratio': 0.22715757417672017, 'model_type': 'GCNConv', 'batch_size': 15, 'lr': 0.008027462923101808, 'clusters': 200, 'sampling': 'nearest_neighbors', 'sampling_k': 25}. Best is trial 0 with value: 0.8818876658327218.


...

 - Metrics: Accuracy=0.9501, F1=0.9194, Recall=0.9413, Precision=0.8985
Done. Results written to cora_experimentations\cora_scar_sampling_nnif_batch_cluster_2803040318.csv.
Average F1 over valid seeds: 0.9119 ± 0.0067
Running experiment with seed=654:
 - K=4, layers=1, hidden=64, out=128
 - norm=None, dropout=0, batch_size=256, methodology=ours
 - ratio=0.25938262754115543, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=25
 - model_type=GCNConv, rate_pairs=1, clusters=300, lr=0.004480872415563464
Epoch 0 / 100, Loss: 10.6947
Epoch 10 / 100, Loss: 7.8931
Epoch 20 / 100, Loss: 5.6721
Epoch 30 / 100, Loss: 4.1495
Epoch 40 / 100, Loss: 3.0018
Epoch 50 / 100, Loss: 2.2835
Epoch 60 / 100, Loss: 1.7759
Epoch 70 / 100, Loss: 1.4255
Epoch 80 / 100, Loss: 1.1883
Epoch 90 / 100, Loss: 0.9848
 - Metrics: Accuracy=0.9420, F1=0.9053, Recall=0.9169, Precision=0.8939
Running experiment with seed=114:
 - K=4, layers=1, hidden=64, out=128
 - norm=N

[I 2025-03-28 04:12:41,431] Trial 198 finished with value: 0.9035419136951311 and parameters: {'K': 4, 'layers': 1, 'hidden_channels': 64, 'out_channels': 128, 'ratio': 0.25938262754115543, 'model_type': 'GCNConv', 'batch_size': 256, 'lr': 0.004480872415563464, 'clusters': 300, 'sampling': 'neighbor', 'sampling_k': 25}. Best is trial 111 with value: 0.9150370188561521.


 - Metrics: Accuracy=0.9383, F1=0.9012, Recall=0.9315, Precision=0.8729
Done. Results written to cora_experimentations\cora_scar_sampling_nnif_batch_cluster_2803040817.csv.
Average F1 over valid seeds: 0.9035 ± 0.0047
Running experiment with seed=654:
 - K=4, layers=1, hidden=64, out=256
 - norm=None, dropout=0, batch_size=15, methodology=ours
 - ratio=0.254081859193035, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=25
 - model_type=GATConv, rate_pairs=1, clusters=400, lr=0.005256282558691244
Epoch 0 / 100, Loss: 160.8666
Epoch 10 / 100, Loss: 5.6128
Epoch 20 / 100, Loss: 1.4399
Epoch 30 / 100, Loss: 0.4975
Epoch 40 / 100, Loss: 0.1918
Epoch 50 / 100, Loss: 0.0757
Epoch 60 / 100, Loss: 0.0305
Epoch 70 / 100, Loss: 0.0129
Epoch 80 / 100, Loss: 0.0051
Epoch 90 / 100, Loss: 0.0021


[I 2025-03-28 04:20:37,928] Trial 199 finished with value: 0.7777031564808596 and parameters: {'K': 4, 'layers': 1, 'hidden_channels': 64, 'out_channels': 256, 'ratio': 0.254081859193035, 'model_type': 'GATConv', 'batch_size': 15, 'lr': 0.005256282558691244, 'clusters': 400, 'sampling': 'neighbor', 'sampling_k': 25}. Best is trial 111 with value: 0.9150370188561521.


 - Metrics: Accuracy=0.8778, F1=0.7777, Recall=0.7078, Precision=0.8629
F1 = 0.78 < 0.85, skipping ...
Done. Results written to cora_experimentations\cora_scar_sampling_nnif_batch_cluster_2803041241.csv.
Average F1 over valid seeds: 0.7777 ± 0.0000
Best trial:
  Average F1: 0.9150370188561521
  Best parameters:
    K: 4
    layers: 1
    hidden_channels: 128
    out_channels: 256
    ratio: 0.26218826843662685
    model_type: GATConv
    batch_size: 256
    lr: 0.00302395227957548
    clusters: 100
    sampling: neighbor
    sampling_k: 25


### Hyperparameter Optimization Cora
#### SAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "cora",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 7, 9),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.16, 0.34),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":trial.suggest_int("rate_pairs", 1, 15),
        "batch_size": trial.suggest_categorical("batch_size", [5,10,15]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "cora_sar_sampling_nnif_batch_cluster.csv",
        "min":0.85,
        "clusters":trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":trial.suggest_categorical("num_epochs", [50,100,150]),         
        "sampling":"cluster",#trial.suggest_categorical("sampling",["sage","cluster","shine"]),         
        
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-03-26 15:51:16,342] A new study created in memory with name: no-name-8176da62-abb1-46e4-8807-5e5ff1459ce0


Running experiment with seed=654:
 - K=9, layers=1, hidden=64, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.16889523261113468, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=75
 - model_type=GCNConv, rate_pairs=1, clusters=400, lr=0.007646163484926425
Epoch 0 / 150, Loss: 3.4303
Epoch 10 / 150, Loss: 3.1501
Epoch 20 / 150, Loss: 2.8808
Epoch 30 / 150, Loss: 2.6330
Epoch 40 / 150, Loss: 2.3545
Epoch 50 / 150, Loss: 2.0481
Epoch 60 / 150, Loss: 1.7211
Epoch 70 / 150, Loss: 1.4545
Epoch 80 / 150, Loss: 1.2300
Epoch 90 / 150, Loss: 1.0373
Epoch 100 / 150, Loss: 0.8788
Epoch 110 / 150, Loss: 0.7530
Epoch 120 / 150, Loss: 0.6579
Epoch 130 / 150, Loss: 0.5790
Epoch 140 / 150, Loss: 0.5190
 - Metrics: Accuracy=0.9228, F1=0.8656, Recall=0.8227, Precision=0.9132
Running experiment with seed=114:
 - K=9, layers=1, hidden=64, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.1688952

[I 2025-03-26 15:53:50,635] Trial 0 finished with value: 0.8684716037566916 and parameters: {'K': 9, 'layers': 1, 'hidden_channels': 64, 'out_channels': 256, 'ratio': 0.16889523261113468, 'model_type': 'GCNConv', 'batch_size': 1024, 'lr': 0.007646163484926425, 'anomaly_detector': 'nearest_neighbors', 'clusters': 400, 'num_epochs': 150, 'sampling': 'neighbor', 'sampling_k': 75}. Best is trial 0 with value: 0.8684716037566916.


 - Metrics: Accuracy=0.9269, F1=0.8731, Recall=0.8325, Precision=0.9178
Done. Results written to cora_experimentations\cora_sar_sampling_nnif_batch_cluster_2603155116.csv.
Average F1 over valid seeds: 0.8685 ± 0.0049
Running experiment with seed=654:
 - K=9, layers=1, hidden=128, out=256
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.1886114414198443, aggregation=mean, treatment=removal, anomaly_detector=unweighted, sampling=nearest_neighbors, sampling_k=25
 - model_type=GCNConv, rate_pairs=1, clusters=200, lr=0.00016096248213070527
Epoch 0 / 150, Loss: 35.3035
Epoch 10 / 150, Loss: 35.6685
Epoch 20 / 150, Loss: 34.8467
Epoch 30 / 150, Loss: 34.7338
Epoch 40 / 150, Loss: 34.8928
Epoch 50 / 150, Loss: 34.1160
Epoch 60 / 150, Loss: 33.9046
Epoch 70 / 150, Loss: 33.9726
Epoch 80 / 150, Loss: 33.1283
Epoch 90 / 150, Loss: 33.2420
Epoch 100 / 150, Loss: 32.6670
Epoch 110 / 150, Loss: 32.7155
[Early Stopping] at epoch 118


[I 2025-03-26 15:54:30,723] Trial 1 finished with value: 0.6825775656324582 and parameters: {'K': 9, 'layers': 1, 'hidden_channels': 128, 'out_channels': 256, 'ratio': 0.1886114414198443, 'model_type': 'GCNConv', 'batch_size': 512, 'lr': 0.00016096248213070527, 'anomaly_detector': 'unweighted', 'clusters': 200, 'num_epochs': 150, 'sampling': 'nearest_neighbors', 'sampling_k': 25}. Best is trial 0 with value: 0.8684716037566916.


...

Epoch 0 / 100, Loss: 5.9769
Epoch 10 / 100, Loss: 5.1580
Epoch 20 / 100, Loss: 4.5073
Epoch 30 / 100, Loss: 3.6713
Epoch 40 / 100, Loss: 3.1895
Epoch 50 / 100, Loss: 2.3100
Epoch 60 / 100, Loss: 1.8741
Epoch 70 / 100, Loss: 1.5376
Epoch 80 / 100, Loss: 1.2265
Epoch 90 / 100, Loss: 1.0520
 - Metrics: Accuracy=0.9210, F1=0.8694, Recall=0.8704, Precision=0.8683
Running experiment with seed=25:
 - K=8, layers=1, hidden=128, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.2573033125681119, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=cluster, sampling_k=25
 - model_type=GCNConv, rate_pairs=1, clusters=100, lr=0.007397697366787508


Computing METIS partitioning...
Done!


Epoch 0 / 100, Loss: 5.8094


[W 2025-03-26 16:49:22,637] Trial 31 failed with parameters: {'K': 8, 'layers': 1, 'hidden_channels': 128, 'out_channels': 256, 'ratio': 0.2573033125681119, 'model_type': 'GCNConv', 'batch_size': 1024, 'lr': 0.007397697366787508, 'anomaly_detector': 'nearest_neighbors', 'clusters': 100, 'num_epochs': 100, 'sampling': 'cluster', 'sampling_k': 25} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\romai\Desktop\gnn\gnn_pu\.conda\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\romai\AppData\Local\Temp\ipykernel_39988\45902056.py", line 39, in objective
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNIF-GNN\train_NNIF_GNN.py", line 565, in run_nnif_gnn_experiment
    train_labels, train_proba, train_losses = train_graph(
          

KeyboardInterrupt: 

### Hyperparameter Optimization Pubmed
#### SCAR 

In [7]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "pubmed",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "K": 12,#trial.suggest_int("K", 2, 20),
        "layers": 2,#trial.suggest_int("layers", 1, 2),
        "hidden_channels": 256,#trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": 256,#trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": 0.24,#trial.suggest_float("ratio", 0.2, 0.4),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": "GATConv",#trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":9,#trial.suggest_int("rate_pairs", 1, 15),
        "batch_size": 15,#trial.suggest_categorical("batch_size", [5,10,15,20]),
        "lr": 0.008,#trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector": "nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 2,
        "output_csv": "pubmed_scar_sampling_nnif_batch_cluster.csv",
        "min":0.84,
        "clusters":100,#trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":100,         
        "sampling":"cluster",#trial.suggest_categorical("sampling",["sage","cluster","shine"]),         
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-04-03 13:37:31,296] A new study created in memory with name: no-name-59d2c2dd-234d-4341-bce1-cafb1a7e5405
Computing METIS partitioning...


Running experiment with seed=654:
 - K=12, layers=2, hidden=256, out=256
 - norm=None, dropout=0, batch_size=15, methodology=ours
 - ratio=0.24, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=cluster
 - model_type=GATConv, rate_pairs=9, clusters=100, lr=0.008


Done!


Epoch 0 / 100, Loss: 8.5803
Epoch 10 / 100, Loss: 7.8900
Epoch 20 / 100, Loss: 5.7751
Epoch 30 / 100, Loss: 3.4927
Epoch 40 / 100, Loss: 2.2187
Epoch 50 / 100, Loss: 1.5352
Epoch 60 / 100, Loss: 1.1733
Epoch 70 / 100, Loss: 0.9024
Epoch 80 / 100, Loss: 0.7083
Epoch 90 / 100, Loss: 0.5843
 - Metrics: Accuracy=0.9002, F1=0.8718, Recall=0.8494, Precision=0.8953
Running experiment with seed=114:
 - K=12, layers=2, hidden=256, out=256
 - norm=None, dropout=0, batch_size=15, methodology=ours
 - ratio=0.24, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=cluster
 - model_type=GATConv, rate_pairs=9, clusters=100, lr=0.008


Computing METIS partitioning...
Done!


Epoch 0 / 100, Loss: 8.7896
Epoch 10 / 100, Loss: 8.2390
Epoch 20 / 100, Loss: 5.6903
Epoch 30 / 100, Loss: 3.5162
Epoch 40 / 100, Loss: 2.1999
Epoch 50 / 100, Loss: 1.5140


[W 2025-04-03 13:39:29,674] Trial 0 failed with parameters: {} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\romai\Desktop\gnn\gnn_pu\.conda\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\romai\AppData\Local\Temp\ipykernel_52060\1944006450.py", line 37, in objective
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNIF-GNN\train_NNIF_GNN.py", line 556, in run_nnif_gnn_experiment
    train_labels, train_proba, train_losses = train_graph(
                                              ^^^^^^^^^^^^
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNIF-GNN\train_NNIF_GNN.py", line 263, in train_graph
    for sub_data in train_loader:
  File "c:\Users\romai\Desktop\gnn\gnn_pu\.conda\Lib\site-packages\torch\utils\data\dataloader.py", line

KeyboardInterrupt: 

### Hyperparameter Optimization Pubmed
#### SAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "pubmed",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 2, 12),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.2, 0.4),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":trial.suggest_int("layers", 1, 15),
        "batch_size": trial.suggest_categorical("batch_size", [15,256,512,1024]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":"nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "pubmed_sar_sampling_nnif_batch_cluster.csv",
        "min":0.84,
        "clusters":trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":trial.suggest_categorical("num_epochs", [50,100,150]),         
        "sampling":trial.suggest_categorical("sampling",["sage","cluster","shine"]),         
        
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-03-24 05:51:18,349] A new study created in memory with name: no-name-7363eccd-5b06-40dc-87b0-25014df8c5a1


Running experiment with seed=654:
 - K=7, layers=2, hidden=128, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.3362908893186948, aggregation=mean, treatment=removal, anomaly_detector=unweighted, sampling=cluster, sampling_k=50
 - model_type=GATConv, rate_pairs=10, clusters=300, lr=0.0007087214622711031


Computing METIS partitioning...
Done!


Epoch 0 / 100, Loss: 18.5522
Epoch 10 / 100, Loss: 17.8089
Epoch 20 / 100, Loss: 17.3418
Epoch 30 / 100, Loss: 16.8934
Epoch 40 / 100, Loss: 16.1184
Epoch 50 / 100, Loss: 15.1700
Epoch 60 / 100, Loss: 14.4180
Epoch 70 / 100, Loss: 13.4842
Epoch 80 / 100, Loss: 12.6036
Epoch 90 / 100, Loss: 11.9384


[I 2025-03-24 05:53:00,820] Trial 0 finished with value: 0.715743932941256 and parameters: {'K': 7, 'ratio': 0.3362908893186948, 'model_type': 'GATConv', 'batch_size': 1024, 'lr': 0.0007087214622711031, 'anomaly_detector': 'unweighted', 'clusters': 300, 'num_epochs': 100, 'sampling': 'cluster', 'sampling_k': 50}. Best is trial 0 with value: 0.715743932941256.


...

 - Metrics: Accuracy=0.8745, F1=0.8391, Recall=0.8193, Precision=0.8598
F1 = 0.84 < 0.84, skipping ...
Done. Results written to pubmed_experimentations\pubmed_sar_sampling_nnif_batch_cluster_2403133828.csv.
Average F1 over valid seeds: 0.8391 ± 0.0000
Running experiment with seed=654:
 - K=7, layers=2, hidden=128, out=256
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.28001597838526016, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=nearest_neighbors, sampling_k=50
 - model_type=GCNConv, rate_pairs=10, clusters=400, lr=0.00873922908266354
Epoch 0 / 50, Loss: 58.0806
Epoch 10 / 50, Loss: 8.0599
Epoch 20 / 50, Loss: 2.8407
Epoch 30 / 50, Loss: 1.4709
Epoch 40 / 50, Loss: 0.8989
 - Metrics: Accuracy=0.8762, F1=0.8413, Recall=0.8212, Precision=0.8624
Running experiment with seed=114:
 - K=7, layers=2, hidden=128, out=256
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.28001597838526016, aggregation=mean, treatmen

[I 2025-03-24 15:01:39,121] Trial 13 finished with value: 0.835498509343777 and parameters: {'K': 7, 'ratio': 0.28001597838526016, 'model_type': 'GCNConv', 'batch_size': 512, 'lr': 0.00873922908266354, 'anomaly_detector': 'nearest_neighbors', 'clusters': 400, 'num_epochs': 50, 'sampling': 'nearest_neighbors', 'sampling_k': 50}. Best is trial 10 with value: 0.8395798210348853.


 - Metrics: Accuracy=0.8702, F1=0.8297, Recall=0.7916, Precision=0.8716
F1 = 0.83 < 0.84, skipping ...
Done. Results written to pubmed_experimentations\pubmed_sar_sampling_nnif_batch_cluster_2403141459.csv.
Average F1 over valid seeds: 0.8355 ± 0.0058
Running experiment with seed=654:
 - K=7, layers=2, hidden=128, out=256
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.26308150498044675, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=nearest_neighbors, sampling_k=50
 - model_type=GCNConv, rate_pairs=10, clusters=400, lr=0.009817622280543212
Epoch 0 / 50, Loss: 57.2382
Epoch 10 / 50, Loss: 6.7150
Epoch 20 / 50, Loss: 2.3497


[W 2025-03-24 15:11:12,619] Trial 14 failed with parameters: {'K': 7, 'ratio': 0.26308150498044675, 'model_type': 'GCNConv', 'batch_size': 512, 'lr': 0.009817622280543212, 'anomaly_detector': 'nearest_neighbors', 'clusters': 400, 'num_epochs': 50, 'sampling': 'nearest_neighbors', 'sampling_k': 50} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\romai\Desktop\gnn\gnn_pu\.conda\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\romai\AppData\Local\Temp\ipykernel_22248\4249684348.py", line 39, in objective
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNIF-GNN\train_NNIF_GNN.py", line 565, in run_nnif_gnn_experiment
    train_labels, train_proba, train_losses = train_graph(
                                              ^^^^^^^^^^^

KeyboardInterrupt: 

### Hyperparameter Optimization WikiCS
#### SCAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "wiki-cs",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 1, 35),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.09, 0.35),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":trial.suggest_int("rate_pairs",1,15),
        "batch_size": trial.suggest_categorical("batch_size", [5,10,15,20]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":"nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "wikics_scar_sampling_nnif_batch_cluster.csv",
        "min":0.88,
        "clusters":trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":100,      
        "sampling":"cluster",#trial.suggest_categorical("sampling",["sage","cluster","shine"]),         
        
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-03-28 04:20:38,026] A new study created in memory with name: no-name-b7804c88-d9f4-4272-be1e-f766b03ffaed
Computing METIS partitioning...


Running experiment with seed=654:
 - K=11, layers=1, hidden=128, out=64
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.1926695943076433, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=cluster, sampling_k=75
 - model_type=GCNConv, rate_pairs=15, clusters=100, lr=0.0025061759908526224


Done!


Epoch 0 / 100, Loss: 18.5554
Epoch 10 / 100, Loss: 17.3968
Epoch 20 / 100, Loss: 14.5902
Epoch 30 / 100, Loss: 13.5468
Epoch 40 / 100, Loss: 11.3363
Epoch 50 / 100, Loss: 10.3032
Epoch 60 / 100, Loss: 9.2580
Epoch 70 / 100, Loss: 8.0010
Epoch 80 / 100, Loss: 6.8279
Epoch 90 / 100, Loss: 5.9414
 - Metrics: Accuracy=0.9435, F1=0.8836, Recall=0.9369, Precision=0.8361
Running experiment with seed=114:
 - K=11, layers=1, hidden=128, out=64
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.1926695943076433, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=cluster, sampling_k=75
 - model_type=GCNConv, rate_pairs=15, clusters=100, lr=0.0025061759908526224


Computing METIS partitioning...
Done!


...

Epoch 0 / 100, Loss: 3.5138
Epoch 10 / 100, Loss: 2.8710
Epoch 20 / 100, Loss: 2.4437
Epoch 30 / 100, Loss: 2.1170
Epoch 40 / 100, Loss: 1.7531
Epoch 50 / 100, Loss: 1.5131
Epoch 60 / 100, Loss: 1.2861
Epoch 70 / 100, Loss: 1.1255
Epoch 80 / 100, Loss: 0.9426
Epoch 90 / 100, Loss: 0.8364
 - Metrics: Accuracy=0.9606, F1=0.9140, Recall=0.9149, Precision=0.9132
Running experiment with seed=281:
 - K=5, layers=2, hidden=256, out=128
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.15148200168671186, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=cluster, sampling_k=75
 - model_type=GCNConv, rate_pairs=14, clusters=50, lr=0.008372583695812601


Computing METIS partitioning...
Done!


Epoch 0 / 100, Loss: 3.2542
Epoch 10 / 100, Loss: 3.0165
Epoch 20 / 100, Loss: 2.9101
Epoch 30 / 100, Loss: 2.1438
Epoch 40 / 100, Loss: 1.7470
Epoch 50 / 100, Loss: 1.5032
Epoch 60 / 100, Loss: 1.2463
Epoch 70 / 100, Loss: 1.2354
Epoch 80 / 100, Loss: 0.9383
Epoch 90 / 100, Loss: 0.8335


[I 2025-03-29 19:17:01,312] Trial 199 finished with value: 0.9171701172307115 and parameters: {'K': 5, 'layers': 2, 'hidden_channels': 256, 'out_channels': 128, 'ratio': 0.15148200168671186, 'model_type': 'GCNConv', 'rate_pairs': 14, 'batch_size': 1024, 'lr': 0.008372583695812601, 'clusters': 50, 'sampling': 'cluster', 'sampling_k': 75}. Best is trial 137 with value: 0.9189642012571827.


 - Metrics: Accuracy=0.9607, F1=0.9128, Recall=0.8988, Precision=0.9272
Done. Results written to wiki-cs_experimentations\wikics_scar_sampling_nnif_batch_cluster_2903191355.csv.
Average F1 over valid seeds: 0.9172 ± 0.0037
Best trial:
  Average F1: 0.9189642012571827
  Best parameters:
    K: 8
    layers: 2
    hidden_channels: 256
    out_channels: 128
    ratio: 0.16224551381027605
    model_type: GCNConv
    rate_pairs: 13
    batch_size: 512
    lr: 0.007865383500608529
    clusters: 50
    sampling: cluster
    sampling_k: 75


### Hyperparameter Optimization WikiCS
#### SAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "wiki-cs",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 3, 40),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.09, 0.18),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":trial.suggest_int("rate_pairs",1,10),
        "batch_size": trial.suggest_categorical("batch_size", [256,512,1024]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":"nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "wikics_sar_sampling_nnif_batch_cluster.csv",
        "min":0.88,
        "clusters":trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":100,         
        "sampling":trial.suggest_categorical("sampling",["sage","cluster","shine"]),         
        
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


# Elliptic Bitcoin

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "elliptic-bitcoin",      
        "mechanism": "SAR2",
        "train_pct": 0.5,
        "alpha": 0.5,
        "K": trial.suggest_int("K", 2, 20),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": trial.suggest_float("dropout", 0, 0.4),
        "ratio": trial.suggest_float("ratio", 0.001, 0.03),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":trial.suggest_int("rate_pairs",1,15),
        "batch_size": trial.suggest_categorical("batch_size", [5,10,15,20]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":"nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 1,
        "output_csv": "elliptic_bitcoin_cluster.csv",
        "min":0.65,
        "clusters":trial.suggest_categorical("clusters", [100,500,1000]),          
        "num_epochs":100,#trial.suggest_categorical("num_epochs", [50,100,150]),         
        "sampling":"cluster",#trial.suggest_categorical("sampling",["neighbor","cluster"]),         
        }

    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")
